In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-03 22:44:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.62MB/s    in 0.2s    

2021-12-03 22:44:28 (5.62 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-D2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [5]:
from pyspark.sql.functions import to_date

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [7]:
# Filter the data to retrieve all the rows where the total_votes count is equal to or greater than 20
helpful_votes_df = vine_df.filter("total_votes>=20")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [8]:
# Filter the data and create a new dataframe where the number of helpful_votes divided by the
# total_votes is equal to or greater than 50%
majority_helpful_df = helpful_votes_df.filter("helpful_votes/total_votes>=0.50")
majority_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [9]:
from pyspark.sql.functions import count
majority_helpful_df.count()

403807

In [10]:
# Filter and create a new dataframe that retrieves all the rows where a review was
# written as part of the Vine program (paid), vine == 'Y'
vine_reviews_df = majority_helpful_df.filter(majority_helpful_df["vine"] == 'Y')
vine_reviews_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [11]:
# Filter and create a new dataframe that retrieves all the rows where the review was
# NOT written as part of the Vine program (unpaid), vine == 'N'
not_vine_reviews_df = majority_helpful_df.filter(majority_helpful_df["vine"] == 'N')
not_vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [12]:
# Total number of helpful vine reviews
vine_review_count = vine_reviews_df.count()
print("Total Number of Helpful Reviews - Paid: %f" % vine_review_count)

# Total number of helpful not vine reviews
not_vine_review_count = not_vine_reviews_df.count()
print("Total Number of Helpful Reviews - Unpaid: %f" % not_vine_review_count)

# Determine the total number of helpful reviews
total_helpful_reviews_count = majority_helpful_df.count()
print("Total Number of Helpful Reviews: %f" % total_helpful_reviews_count)

Total Number of Helpful Reviews - Paid: 0.000000
Total Number of Helpful Reviews - Unpaid: 403807.000000
Total Number of Helpful Reviews: 403807.000000


In [13]:
# Determine the number of helpful 5-star reviews
five_star_reviews_count = majority_helpful_df.filter(majority_helpful_df["star_rating"] == '5').count()
print("The total number of helpful five star reviews: %f" % five_star_reviews_count)

The total number of helpful five star reviews: 242889.000000


In [14]:
# Determine the number of 5-star reviews for vine and non vine reviews
vine_five_star_reviews_count = vine_reviews_df.filter(vine_reviews_df["star_rating"] == '5').count()
print("Total Number of Vine Five Star Reviews - Paid and Helpful: %f" % vine_five_star_reviews_count)

not_vine_five_star_reviews_count = not_vine_reviews_df.filter(not_vine_reviews_df["star_rating"] == '5').count()
print("Total Number of Not Vine Five Star Reviews - Unpaid and Helpful: %f" % not_vine_five_star_reviews_count)

Total Number of Vine Five Star Reviews - Paid and Helpful: 0.000000
Total Number of Not Vine Five Star Reviews - Unpaid and Helpful: 242889.000000


In [22]:
# Determine the percentage of 5-star reviews for the two types of reviews (paid vs unpaid)

# The percentage of PAID 5-star reviews
fivestar_vine_pct = (vine_five_star_reviews_count / total_helpful_reviews_count)*100
print("The Percentage of Vine Five Star Reviews - Paid and Helpful: %f" % fivestar_vine_pct)

# The percentage of NOT PAID 5-star reviews
fivestar_not_vine_pct = (not_vine_five_star_reviews_count / total_helpful_reviews_count)*100
print("The Percentage of Not Vine Five Star Reviews - Unpaid and Helpful: %f" % fivestar_not_vine_pct)

The Percentage of Vine Five Star Reviews - Paid and Helpful: 0.000000
The Percentage of Not Vine Five Star Reviews - Unpaid and Helpful: 60.149775


In [16]:
# Filter the dataframe to determine how many vine reviews exist in the dataset
total_vine_reviews_df = df.filter(df["vine"] == 'Y')
total_vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42321535|R2NKNL4WXWIWBG|1400062217|     791531418|The Laments: A Novel|           Books|          4|            3|          3|   Y|                Y|Doh! He *almost* ...|There were times ...| 2005-09-12|
|         US|   52254603| RAIG9ZQDMBF5P|0679742034|     586032452|    Buffalo Soldiers|           Books|          5|    

In [17]:
# Find the total number of reviews in this dataset
total_review_count = df.count()
total_review_count

3105520

In [18]:
# Find the total number of vine reviews in this dataset
total_vine_reviews_df.count()

2

In [19]:
# Find the total number of non vine reviews in this dataset
not_vine_reviews_df = df.filter(df["vine"] == 'N')
not_vine_reviews_df.count()

3105513

In [20]:
# Find the total number of non vine reviews that were 5 stars
all_five_star_novine = not_vine_reviews_df.filter(not_vine_reviews_df["star_rating"] == '5')
all_five_star_novine.count()

1864803

In [21]:
# Determine the total number of 5-star reviews
all_five_star_reviews_count = df.filter(df["star_rating"] == '5').count()
print("The total number of five star reviews: %f" % all_five_star_reviews_count)

The total number of five star reviews: 1864804.000000
